In [2]:
# check python version
import sys
print(sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [3]:
# notebook settings

COLAB_MODE = False # set to True if running in Google Colab
ENABLE_JSON2CSV = False # set to True if you want to convert json dataset to csv

In [4]:
# if COLAB_MODE is True, then work around the repository
if COLAB_MODE:
    import os
    branch_name = 'dhupee-dev'
    cloned_repo_name = 'remote-clone'
    target_repo_dir = '/content/remote-clone/ML'
    repo_link = 'https://github.com/dhupee/Bangkit-C22CB-Company-Based-Capstone.git'
    # if current directory is not the cloned repo, clone it
    if not os.path.exists(target_repo_dir):
        !git clone --single-branch --branch $branch_name $repo_link $cloned_repo_name
        print('Repo successfully cloned!')
        %cd $target_repo_dir
        %pwd
    else:
        print('Repo already cloned')

In [5]:
# check if transformers and tensorflow are installed, if not install them
# use transformers version 4.18.0 and tensorflow version 2.8.0
try:
    import transformers
    import tensorflow as tf
    print("transformers and tensorflow are installed")
except:
    print("transformers and tensorflow are not installed")
    print("installing transformers and tensorflow")
    # install transformers 4.18.0 and tensorflow 2.8.0
    %pip install transformers==4.18.0
    %pip install tensorflow==2.8.0
    # import transformers and tensorflow again
    import transformers
    import tensorflow as tf

d:\DH\projects\Bangkit-C22CB-Company-Based-Capstone\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers and tensorflow are installed


In [6]:
model_name = "cahya/bert-base-indonesian-522M"
batch_size = 32

from transformers import BertTokenizer, TFAutoModel # make sure use tensorflow model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
Total params: 110,617,344
Trainable params: 110,617,344
Non-trainable params: 0
_________________________________________________________________


In [8]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizer)

In [9]:
# test tokenizer
tokenizer("Nama kamu siapa?")

{'input_ids': [3, 1769, 8343, 6186, 32, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer("saya suka makan nasi goreng")

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


{'input_ids': [3, 3245, 5366, 2464, 6014, 11186, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [11]:
unmasker = transformers.pipeline('fill-mask', model = model_name)
unmasker("mainan saya [MASK] di jalan")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


[{'score': 0.0840364545583725,
  'token': 2186,
  'token_str': 'berada',
  'sequence': 'mainan saya berada di jalan'},
 {'score': 0.07038316130638123,
  'token': 1821,
  'token_str': 'ada',
  'sequence': 'mainan saya ada di jalan'},
 {'score': 0.0403575636446476,
  'token': 1998,
  'token_str': 'sendiri',
  'sequence': 'mainan saya sendiri di jalan'},
 {'score': 0.029048316180706024,
  'token': 2444,
  'token_str': 'lahir',
  'sequence': 'mainan saya lahir di jalan'},
 {'score': 0.028137197718024254,
  'token': 3812,
  'token_str': 'berdiri',
  'sequence': 'mainan saya berdiri di jalan'}]

In [12]:
# load dataset json file
import json

train_json_dir = "Translated/train-v2.0_indo.json"
dev_json_dir = "Translated/dev-v2.0_indo.json"
tester_json_dir  = "Translated/tester_indo.json"

dataset_dirs = [train_json_dir, dev_json_dir, tester_json_dir]
# dataset_dirs = [tester_json_dir]

In [13]:
'''
    This function is for converting SQuAD json file to pandas dataframe, iteratively

    I dont want run this locally, better use colab
'''

if ENABLE_JSON2CSV:
    import utils
    for dir in dataset_dirs:
        with open(dir, encoding="utf-8") as json_file:
            file = json.load(json_file)
            dict_file = file
            data = dict_file['data']

        df = utils.json_to_df(data)
        df.to_csv(dir.replace(".json", ".csv"), index = False)

---

In [14]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

In [15]:
try:
    import pandas as pd
except:
    print("pandas is not installed")
    print("installing pandas...")
    %pip install pandas
    import pandas as pd

train_csv_dir = "Translated/train-v2.0_indo.csv"
dev_csv_dir = "Translated/dev-v2.0_indo.csv"
tester_csv_dir  = "Translated/tester_indo.csv"

# open one of the csv file
df_tester = pd.read_csv(tester_csv_dir)
df_tester.head()

,input_id,title,context,question,answer_text,answer_start
0,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
1,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
2,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
3,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
4,56ddde6b9a695914005b9629,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Kapan orang Normandia di Normandia?,abad 10 dan 11,94.0


In [60]:
#check dataframe datatype
df_tester.dtypes

input_id         object
title            object
context          object
question         object
answer_text      object
answer_start    float64
dtype: object

In [33]:
csv_dirs = [train_csv_dir, dev_csv_dir, tester_csv_dir]

# check every csv file, if there is feature longer than max_length, then print it
for csv_dir in csv_dirs:
    df = pd.read_csv(csv_dir)
    longer_feature = 0
    for index, row in df.iterrows():
        if len(row[['context', 'question', 'input_id']]) > max_length:
            longer_feature += 1
            print(f"{csv_dir} has {longer_feature} features longer than {max_length}")
            print("\n")
    if longer_feature == 0:
        print(f"{csv_dir} has no features longer than {max_length}")
        print("\n")

Translated/train-v2.0_indo.csv has no features longer than 384


Translated/dev-v2.0_indo.csv has no features longer than 384


Translated/tester_indo.csv has no features longer than 384




In [35]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
learning_rate = 2e-5
num_train_epochs = 2
weight_decay = 0.01

In [38]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [57]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    
    pad_on_right = tokenizer.padding_side == "right"

    tokenized_examples = tokenizer(
        examples['context'],
        examples['question'],
        add_special_tokens=True, # add [CLS], [SEP]
        max_length=max_length, # max length of feature
        stride=doc_stride, # allowed overlap between two part of the context when splitting is performed
        truncation_strategy='only_second', # truncate the first part of the context
        padding='max_length', # padding strategy
        return_tensors="tf" # return tensorflow tensor
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [58]:
features = prepare_train_features(df_tester)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# TODO: load preprocessed dataset
# TODO: train model
# TODO: evaluate model
# TODO: save model